# **ENTRENAMIENTO DE DETECCIÓN DE MASCARILLA**





In [1]:
# Importar e intalar TF1
%tensorflow_version 1.x
!pip install tf_slim
!pip show tensorflow

TensorFlow 1.x selected.
     |████████████████████████████████| 358kB 8.3MB/s 
Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python3.7
Requires: tensorboard, opt-einsum, protobuf, termcolor, six, gast, keras-preprocessing, grpcio, wheel, google-pasta, astor, wrapt, tensorflow-estimator, absl-py, keras-applications, numpy
Required-by: stable-baselines, magenta, fancyimpute


## **0. Crear directorios**

In [2]:
from google.colab import drive
import os

drive.mount('carga')

Mounted at carga


Verificar conexión a GPU


In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU no encontrada')
print('Encontrada GPU: {}'.format(device_name))

Encontrada GPU: /device:GPU:0


Como primer paso creamos los directorios que nos ayudarán a tener organizado nuestro entorno de trabajo.

In [4]:
!mkdir /content/workspace
!mkdir /content/workspace/MASK
!mkdir /content/workspace/MASK/pre-trained-models
!mkdir /content/workspace/MASK/models

## **1. Configuración de rutas**

Se configuran las rutas necesarias para faciltar su cambio en todo el código de ser necesario. 

In [5]:
CUSTOM_MODEL_NAME = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19'
#CUSTOM_MODEL_NAME = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18'

APIMODEL_PATH = 'models'
WORKSPACE_PATH = 'workspace/MASK'
MODEL_PATH = WORKSPACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
CHECKPOINT_PATH = MODEL_PATH + '/' + CUSTOM_MODEL_NAME

IMAGE_PATH = '/content/carga/MyDrive/workspace/images-todo'
ANNOTATION_PATH = '/content/carga/MyDrive/workspace/annotations' 
OUTPUT_PATH = '/content/carga/MyDrive/workspace/tf1/35-V-'+ CUSTOM_MODEL_NAME + '/output'
EXPORT_PATH = '/content/carga/MyDrive/workspace/tf1/35-V-'+ CUSTOM_MODEL_NAME + '/export'

## **3. Instalar la API de detección de objetos de TensorFlow**

In [6]:
import os
import pathlib

# Clonar el repositorio de TensorFlow
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !apt-get install protobuf-compiler
  !git clone https://github.com/tensorflow/models.git

os.environ['PYTHONPATH'] += ':/content/models/research/'
os.environ['PYTHONPATH'] += ':/content/models/research/slim/'
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection/utils/'
os.environ['PYTHONPATH'] += ':/content/models/research/object_detection'

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
Cloning into 'models'...
remote: Enumerating objects: 1250, done.
remote: Counting objects: 100% (1250/1250), done.
remote: Compressing objects: 100% (679/679), done.
remote: Total 55242 (delta 682), reused 1038 (delta 553), pack-reused 53992
Receiving objects: 100% (55242/55242), 572.29 MiB | 32.57 MiB/s, done.
Resolving deltas: 100% (37766/37766), done.


In [7]:
%%bash
cd models/research
pip install --upgrade pip
# Compilar protos
protoc object_detection/protos/*.proto --python_out=.
# Instalar la API TensorFlow Object Detection
cp object_detection/packages/tf1/setup.py .
python -m pip install .

  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1643258 sha256=b58aaa5cb49c1112e5504f148fa17c6553cf9ba1ab12858f29d99609f5be35a7
  Stored in directory: /tmp/pip-ephem-wheel-cache-7njniwkx/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
Successfully built object-detection


In [8]:
!python models/research/object_detection/builders/model_builder_tf1_test.py

Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_w

## **4. Descargar el modelo TF previamente entrenado**

In [9]:
%cd /content/
!wget {'http://download.tensorflow.org/models/object_detection/' + CUSTOM_MODEL_NAME + '.tar.gz'}
!mv {CUSTOM_MODEL_NAME+'.tar.gz'} {PRETRAINED_MODEL_PATH}
!cd {PRETRAINED_MODEL_PATH} && tar -zxvf {CUSTOM_MODEL_NAME+'.tar.gz'}

/content
--2021-04-05 21:48:17--  http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 156413934 (149M) [application/x-tar]
Saving to: ‘ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’

ssdlite_mobiledet_e 100%[===================>] 149.17M   169MB/s    in 0.9s    

2021-04-05 21:48:18 (169 MB/s) - ‘ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz’ saved [156413934/156413934]

ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.data-00000-of-00001
ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/fp32/model.ckpt-400000.index
ssdlite_mobil

In [10]:
!mkdir {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH + '/' + CUSTOM_MODEL_NAME + '/uint8/pipeline.config'} {MODEL_PATH + '/' + CUSTOM_MODEL_NAME}

## **7. Actualizar la configuración para el aprendizaje por transferencia**

In [12]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

CONFIG_PATH = CHECKPOINT_PATH + '/pipeline.config'
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 

pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 30
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH +'/'+ CUSTOM_MODEL_NAME + '/uint8/model.ckpt'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']
# Entrenamiento conciente de cuantización
pipeline_config.graph_rewriter.quantization.delay = 15000
pipeline_config.graph_rewriter.quantization.weight_bits = 8
pipeline_config.graph_rewriter.quantization.activation_bits = 8

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [13]:
# ver la GPU usada
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)
# Archivo de confguración modificado
!cat $CONFIG_PATH

Mon Apr  5 21:48:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    28W /  70W |    104MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **8. Entrenar modelo**

TensorBoard

In [ ]:
%cd /content
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
# Comenzar TensorBoard
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format('/content/carga/MyDrive/workspace/tf1/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/output')
)
get_ipython().system_raw('./ngrok http 6006 &')
print('Click on link below to track progress:')
import time
time.sleep(1)
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Entrenamiento del modelo



In [14]:
!python $APIMODEL_PATH/research/object_detection/model_main.py \
    --pipeline_config_path=$CHECKPOINT_PATH/pipeline.config \
    --model_dir=$OUTPUT_PATH \
    --num_train_steps=50000 \
    --num_eval_steps=1000 \
    --logtostderr=true

Se han truncado las últimas 5000 líneas del flujo de salida.
INFO:tensorflow:Skipping quant after FeatureExtractor/MobileDetEdgeTPU/IBN_10/SeparableConv2d/add_fold
I0405 23:40:37.846961 140559175210880 quantize.py:299] Skipping quant after FeatureExtractor/MobileDetEdgeTPU/IBN_10/SeparableConv2d/add_fold
INFO:tensorflow:Skipping quant after FeatureExtractor/MobileDetEdgeTPU/C5_1_Conv2d_2_1x1_256/add_fold
I0405 23:40:37.847328 140559175210880 quantize.py:299] Skipping quant after FeatureExtractor/MobileDetEdgeTPU/C5_1_Conv2d_2_1x1_256/add_fold
INFO:tensorflow:Skipping quant after FeatureExtractor/MobileDetEdgeTPU/C5_2_Conv2d_2_3x3_s2_512_depthwise/add_fold
I0405 23:40:37.847588 140559175210880 quantize.py:299] Skipping quant after FeatureExtractor/MobileDetEdgeTPU/C5_2_Conv2d_2_3x3_s2_512_depthwise/add_fold
INFO:tensorflow:Skipping quant after FeatureExtractor/MobileDetEdgeTPU/C5_2_Conv2d_2_3x3_s2_512/add_fold
I0405 23:40:37.847830 140559175210880 quantize.py:299] Skipping quant after F

Evaluar modelo pepe


In [15]:
!python $APIMODEL_PATH/research/object_detection/model_main.py \
  --model_dir=$OUTPUT_PATH \
  --pipeline_config_path=$CHECKPOINT_PATH/pipeline.config \
  --checkpoint_dir=$OUTPUT_PATH

Output hidden; open in https://colab.research.google.com to view.

## **9. Exportar el modelo desde el punto de control** 

Exportar el modelo con MODEL_MAIN_TF2

In [16]:
!python $APIMODEL_PATH/research/object_detection/export_inference_graph.py \
  --input_type=image_tensor --pipeline_config_path=$CHECKPOINT_PATH/pipeline.config \
  --output_directory=$EXPORT_PATH/export_main_tf1 \
  --trained_checkpoint_prefix=$OUTPUT_PATH/model.ckpt-50000

Instructions for updating:
Please use `layer.__call__` method instead.
W0406 03:08:23.802064 140348297086848 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:08:26.004979 140348297086848 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:08:26.093106 140348297086848 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:08:26.171980 140348297086848 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

Exportar modelo compatible con TFLite Coverter


In [17]:
!python $APIMODEL_PATH/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$CHECKPOINT_PATH/pipeline.config \
    --trained_checkpoint_prefix=$OUTPUT_PATH/model.ckpt-50000 \
    --output_directory=$EXPORT_PATH/export_graph_tf1_2 \
    --output_format=TFLITE --input_shapes=1,320,320,3 \
    --input_arrays=normalized_input_image_tensor \
    --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' \
    --allow_custom_ops --inference_type=QUANTIZED_UINT8 --mean_values=128 --std_dev_values=127 --change_concat_input_ranges=false --allow_custom_ops \
    --add_postprocessing_op=true

Instructions for updating:
Please use `layer.__call__` method instead.
W0406 03:08:44.198598 139907520751488 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:08:46.389990 139907520751488 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:08:46.465087 139907520751488 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:08:46.536854 139907520751488 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

In [18]:
!python $APIMODEL_PATH/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path=$CHECKPOINT_PATH/pipeline.config \
    --trained_checkpoint_prefix=$OUTPUT_PATH/model.ckpt-50000 \
    --output_directory=$EXPORT_PATH/export_graph_tf1 \
    --inference_type=QUANTIZED_UINT8 \
    --add_postprocessing_op=true


Instructions for updating:
Please use `layer.__call__` method instead.
W0406 03:08:58.647010 140450479589248 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:09:00.830528 140450479589248 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:09:00.902709 140450479589248 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0406 03:09:00.974537 140450479589248 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

# **Inferencia (Export inference graph)**

In [19]:
# Pequeña evaluacióón del modelo
import numpy as np
import os
import sys
import tensorflow as tf

from collections import defaultdict
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
%matplotlib inline

# Inicializar tf.Graph()
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile('/content/carga/MyDrive/workspace/tf1/35-V-ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19/export/export_main_tf1/frozen_inference_graph.pb', 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

# Cargar etiquetas
label_map = label_map_util.load_labelmap(ANNOTATION_PATH + '/label_map.pbtxt')
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=2, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# ejecuta la Inferencia y rellena los resultados en un dict.
def run_inference(graph, image):
  with graph.as_default():
    with tf.Session() as sess:
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = [output.name for op in ops for output in op.outputs]
      tensor_dict = {}
      tensor_keys = ['num_detections', 'detection_boxes', 'detection_scores', 'detection_classes']
      for key in tensor_keys:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
      
      # Actual inferencia.
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
      output_dict = sess.run(tensor_dict, feed_dict={image_tensor: np.expand_dims(image, 0)})

      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
  return output_dict

test_image_path = [os.path.join('/content/carga/MyDrive/workspace/test/', 'image{}.png'.format(i)) for i in range(1, 53)]
for image_path in test_image_path:
  print('Evaluating:', image_path)
  image = Image.open(image_path)
  img_width, img_height = image.size
  image_np = np.array(image.getdata()).reshape((img_height, img_width, 3)).astype(np.uint8)
  # Correr inferencia
  output_dict = run_inference(detection_graph, image_np)
  # Visualización 
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=(12, 8))
  plt.imshow(image_np)

Output hidden; open in https://colab.research.google.com to view.